# Laboratório 2 - Ajuste fino de um modelo de IA generativo para resumo de diálogos

### Configurar o Kernel, baixar as dependências, Dataset e LLM

In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
  transformers==4.27.2 \
  datasets==2.11.0 \
  evaluate==0.4.0 \
  rouge_score==0.1.2 \
  loralib==0.1.1 \
  peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.17.1 requires torch

In [2]:
#Importanto as bibliotecas
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig,TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [3]:
# importando o dataset a partir do huggingface
huggingface_dataset_name = 'knkarthick/dialogsum'
dataset = load_dataset(huggingface_dataset_name)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [4]:
# baixando o modelo pretreinado e o tokenizador
model_name = 'google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name,torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
# número de parametros treináveis no modelo
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _,param in model.named_parameters():
      all_model_params += param.numel()
      if param.requires_grad:
        trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.0%


### 1.3 Testandoo modelo com Zero Shot Inference

In [6]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
      original_model.generate(
          inputs["input_ids"],
          max_new_tokens=200,
      )[0],
      skip_special_tokens=True
  )

dash_line = ('-'.join('' for x in range(100)))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------

# 2- Fazendo o Full Fine_Tuning

### Pré-processando o dataset 'Dialog-Summary'

In [7]:
# coloca um prompt em todo o dataset
def tokenize_function(example):
  start_prompt = 'Summarize the following conversation.\n\n'
  end_prompt = '\n\nSummary: '
  prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
  example['input_ids'] = tokenizer(prompt, padding='max_length', truncation=True, return_tensors='pt').input_ids
  example['labels'] = tokenizer(example['summary'], padding='max_length', truncation=True, return_tensors='pt').input_ids

  return example

# dataset possiu 3 divisões: train, test, validation
# o código tokenize_function está manipulando todos os dados em todas as divisões em batches
# map(): Este é um método que aplica uma função específica a cada elemento do conjunto de dados, no caso o prompt.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id','topic','dialogue','summary'])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [8]:
# Este código é usado para filtrar o conjunto de dados tokenizado para manter apenas cada 100º exemplo.
# Isso pode ser útil para reduzir o tamanho do conjunto de dados ou para criar um subconjunto de dados para fins de depuração ou teste.
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index %100 == 0, with_indices=True)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
print(f'Shapes of the datasets: ')
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets: 
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
})


### 2.2 Fine-Tune o modelo com o dataset pré-processado

In [10]:
from types import SimpleNamespace
from pathlib import Path
from tqdm.notebook import tqdm
from datetime import datetime

In [11]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

config = SimpleNamespace(
    # hyperparameters
    learning_rate=1e-5,
    gradient_accumulation_steps=16,
    num_train_epochs=1, # modificado para '1'
    save_steps=1000,
    save_strategy='steps',
    eval_steps=1000,
    logging_steps=1000,
    evaluation_strategy="steps",
    warmup_steps=500,
    save_total_limit=3,
    load_best_model_at_end = True,
    per_device_train_batch_size= 1, # adicionado
    per_device_eval_batch_size=1, # adicionado
    output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'
)

In [13]:
# Cria um nome de diretório único para salvar os arquivos de treinamento
# com o número no final variando dependendo do tempo atual.

# Isso é útil para evitar a sobrescritura de arquivos de treinamento anteriores
# quando você executa o treinamento várias vezes.

training_args = TrainingArguments(
    output_dir=config.output_dir,
    learning_rate=config.learning_rate,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    num_train_epochs=config.num_train_epochs,
    save_steps=config.save_steps,
    save_strategy=config.save_strategy,
    eval_steps=config.eval_steps,
    logging_steps=config.logging_steps,
    evaluation_strategy=config.evaluation_strategy,
    warmup_steps=config.warmup_steps,
    save_total_limit=config.save_total_limit,
    load_best_model_at_end=config.load_best_model_at_end,
    per_device_train_batch_size=config.per_device_train_batch_size,  # Adicionado
    per_device_eval_batch_size=config.per_device_eval_batch_size,    # Adicionado
    run_name=f"Prompt_tuning_original_model-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)


In [14]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=7, training_loss=49.479910714285715, metrics={'train_runtime': 42.6827, 'train_samples_per_second': 2.929, 'train_steps_per_second': 0.164, 'total_flos': 76692823474176.0, 'train_loss': 49.479910714285715, 'epoch': 0.9})

In [15]:
# é usado para acessar o melhor checkpoint do modelo durante ou após o treinamento
# de um modelo usando a biblioteca transformers da Hugging Face.

trainer.state.best_model_checkpoint

In [16]:
# Salva o modelo atual do treinador no novo diretório flan-dialogue-summary-checkpoint
# Isso é útil se você quiser fazer um backup do seu modelo ou se quiser usar o mesmo modelo em um script diferente mais tarde.

trainer.state.best_model_checkpoint
!mkdir flan-dialogue-summary-checkpoint
custom_path = "./flan-dialogue-summary-checkpoint/"
trainer.save_model(output_dir=custom_path)

In [17]:
# Lista todos os arquivos e diretórios no diretório incluindo arquivos ocultos,
# e está mostrando detalhes adicionais sobre cada arquivo.
# Isso é útil para ver o que está no diretório e para obter informações sobre os arquivos.
!ls -al ./flan-dialogue-summary-checkpoint/

total 483668
drwxr-xr-x 2 root root      4096 Mar 19 12:33 .
drwxr-xr-x 1 root root      4096 Mar 19 12:33 ..
-rw-r--r-- 1 root root      1529 Mar 19 12:33 config.json
-rw-r--r-- 1 root root       142 Mar 19 12:33 generation_config.json
-rw-r--r-- 1 root root 495253173 Mar 19 12:33 pytorch_model.bin
-rw-r--r-- 1 root root      3643 Mar 19 12:33 training_args.bin


In [18]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-dialogue-summary-checkpoint", torch_dtype=torch.bfloat16)

In [19]:
instruct_model = instruct_model.to("cuda")

### Avaliando o modelo qualitativamente (valiação humana)

In [22]:
lindex = [100,200,300]
for index in lindex:
  dialogue = dataset['test'][index]['dialogue']
  human_baseline_summary = dataset['test'][index]['summary']

  prompt = f"""
  Summarize the following conversation.

  {dialogue}

  Summary:
  """

  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  input_ids = input_ids.to("cuda")
  original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
  original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

  instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
  instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
  print(dash_line)
  print(f'ORIGINAL MODEL:\n{original_model_text_output}')
  print(dash_line)
  print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
  print(dash_line)

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# and Mike have a disagreement on how to act out a scene. #Person1# proposes that Mike can try to act in #Person1#'s way.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
The person who sat down with the other person was upset.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
The two of them will try to figure out how to express their feelings.
---------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------

### Avaliação do modelo quantitativamente com a métrica ROUGE

In [23]:
rouge= evaluate.load('rouge')

In [37]:
# gerando outputs para exemplos do dataset(10 diálogos e sumários)
dialogue= dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

# Gerando resumos com dois modelos diferentes - original_model e instruct_model.
# Ele faz isso criando um prompt de resumo para o diálogo, codificando esse prompt em IDs de entrada,
# e então passando esses IDs de entrada para cada modelo para gerar um resumo.
original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogue):
  prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

# Os resumos gerados pelos modelos são então decodificados de volta para o texto e adicionados a duas listas separadas
# uma para os resumos gerados pelo original_model e outra para os resumos gerados pelo instruct_model.
  inputs_ids = tokenizer(prompt, return_tensors='pt').input_ids

  original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
  original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
  original_model_summaries.append(original_model_text_output)

  instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
  instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
  instruct_model_summaries.append(instruct_model_text_output)

# Cria um DataFrame do pandas que contém os resumos gerados por humanos (para comparação), bem como os resumos gerados pelos dois modelos.
# Cada linha do DataFrame corresponde a um diálogo, e as colunas contêm o resumo gerado por humanos, o resumo gerado pelo original_model e o resumo gerado pelo instruct_model.
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))

df= pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,#Person1#: I am proud to say that I am our Pre...,The President of the United States is a man of...
1,In order to prevent employees from wasting tim...,Person1: I cannot imagine if Trump was to be o...,The President of the United States is a man of...
2,Ms. Dawson takes a dictation for #Person1# abo...,The President of the US is a person who has no...,The President of the United States is a man of...
3,#Person2# arrives late because of traffic jam....,#Person1: I can't imagine if Trump were to be ...,The President of the United States is a man of...
4,#Person2# decides to follow #Person1#'s sugges...,#Person1#: I'm proud to say that he is our Pre...,The President of the United States is a man of...
5,#Person2# complains to #Person1# about the tra...,Person1: I have nothing but faith in Trump. Pe...,The President of the United States is a man of...
6,#Person1# tells Kate that Masha and Hero get d...,The president's campaign is preparing a speech...,The President of the United States is a man of...
7,#Person1# tells Kate that Masha and Hero are g...,#Person1#: I cannot imagine if Trump were to b...,The President of the United States is a man of...
8,#Person1# and Kate talk about the divorce betw...,#Person1: I don't think he's the right person....,The President of the United States is a man of...
9,#Person1# and Brian are at the birthday party ...,: I have nothing but faith in Trump. #Person1#...,The President of the United States is a man of...


In [38]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL')
print(original_model_results)
print('INSTRUCT MODEL')
print(instruct_model_results)

ORIGINAL MODEL
{'rouge1': 0.11491056478593781, 'rouge2': 0.0, 'rougeL': 0.0872827079896405, 'rougeLsum': 0.08635076968427241}
INSTRUCT MODEL
{'rouge1': 0.11104176347958539, 'rouge2': 0.0, 'rougeL': 0.08653859564353603, 'rougeLsum': 0.08744227529721565}


In [39]:
# results = pd.read_csv('/content/dialogue-summary-training-1710851527')

# human_baseline_summaries = results['human_baseline_summaries'].value
# original_baseline_summaries = results['original_baseline_summaries'].value
# instuct_baseline_summaries = results['instruct_baseline_summaries'].value

# original_model_results = rouge.compute(
#     predictions=original_model_summaries,
#     references=human_baseline_summaries[0:len(original_model_summaries)],
#     use_aggregator=True,
#     use_stemmer=True,
# )

# instruct_model_results = rouge.compute(
#     predictions=instruct_model_summaries,
#     references=human_baseline_summaries[0:len(instruct_model_summaries)],
#     use_aggregator=True,
#     use_stemmer=True,
# )

# print('ORIGINAL MODEL')
# print(original_model_results)
# print('INSTRUCT MODEL')
# print(instruct_model_results)

human_baseline_summaries = df['human_baseline_summaries'].values
original_model_summaries = df['original_model_summaries'].values
instruct_model_summaries = df['instruct_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.11491056478593781, 'rouge2': 0.0, 'rougeL': 0.0872827079896405, 'rougeLsum': 0.08635076968427241}
INSTRUCT MODEL:
{'rouge1': 0.11104176347958539, 'rouge2': 0.0, 'rougeL': 0.08653859564353603, 'rougeLsum': 0.08744227529721565}


In [45]:
print('Porcentagem absoluta de melhora entre INSTRUCT MODEL  e HUMAN BASELINE')

improvement =(np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
  print(f'{key}:  {value*100:.2f}%')

Porcentagem absoluta de melhora entre INSTRUCT MODEL  e HUMAN BASELINE
rouge1:  -0.39%
rouge2:  0.00%
rougeL:  -0.07%
rougeLsum:  0.11%


# Perform Parameter Efficient Fine_tuning(PEFT)

### Configurando o PEFT/LoRA model para Fine-Tuning

In [46]:
from peft import LoraConfig, get_peft_model, TaskType

In [47]:
config2 = SimpleNamespace(
    # hyperparameters
    learning_rate=1e-3,
    gradient_accumulation_steps=2,
   num_train_epochs=1, # modificado para '1'
    save_steps=1000,
    save_strategy='steps',
    eval_steps=1000,
    logging_steps=1000,
    evaluation_strategy="steps",
    warmup_steps=500,
    save_total_limit=3,
    load_best_model_at_end = True,
    output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}',
    r=32,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM, # FLAN-T5
    auto_find_batch_size=True,
    per_device_train_batch_size= 1, # adicionado
    per_device_eval_batch_size=1, # adicionado
)

In [48]:
lora_config = LoraConfig(
    r=config2.r,
    lora_alpha=config2.lora_alpha,
    target_modules=config2.target_modules,
    lora_dropout=config2.lora_dropout,
    bias=config2.bias,
    task_type=config2.task_type # FLAN-T5
)

In [49]:
# mostrando a quantidade de parametros que serão realmente treinados com o Lora_PEFT
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.4092820552029972%


### 3.2 Train PEFT Adapter

In [51]:
# Define training arguments e cria as instâncias do Trainer

output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    auto_find_batch_size=config2.auto_find_batch_size,
    output_dir=config2.output_dir,
    learning_rate=config2.learning_rate,
    gradient_accumulation_steps=config2.gradient_accumulation_steps,
    num_train_epochs=config2.num_train_epochs,
    save_steps=config2.save_steps,
    save_strategy=config2.save_strategy, # we cannot set it to "no". Otherwise, the model cannot guess the best checkpoint.
    eval_steps=config2.eval_steps,
    logging_steps=config2.logging_steps,
    evaluation_strategy=config2.evaluation_strategy,
    warmup_steps=config2.warmup_steps,
    save_total_limit=config2.save_total_limit,
    load_best_model_at_end = config2.load_best_model_at_end,
    run_name=f"PEFT_tuning_original_model-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"


)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets['validation']
)

In [52]:
peft_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=16, training_loss=50.0390625, metrics={'train_runtime': 38.6932, 'train_samples_per_second': 3.231, 'train_steps_per_second': 0.414, 'total_flos': 86953623552000.0, 'train_loss': 50.0390625, 'epoch': 1.0})

In [53]:
peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/spiece.model',
 './peft-dialogue-summary-checkpoint-local/added_tokens.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [54]:
peft_trainer.state.best_model_checkpoint

In [55]:
custom_path = "./peft-dialogue-summary-checkpoint/"
peft_trainer.save_model(output_dir=custom_path)

In [56]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       peft_model_path,
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [57]:
peft_model = peft_model.to("cuda")

In [60]:
lindex = [200]
for index in lindex:
  dialogue = dataset['test'][index]['dialogue']
  baseline_human_summary = dataset['test'][index]['summary']

  prompt = f"""
  Summarize the following conversation.

  {dialogue}

  Summary: """

  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  input_ids = input_ids.to("cuda")
  original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
  original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

  instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
  instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

  peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
  peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
  print(dash_line)
  print(f'ORIGINAL MODEL:\n{original_model_text_output}')
  print(dash_line)
  print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
  print(dash_line)
  print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is crazy for Trump and voted for him. #Person2# doesn't agree with #Person1# on Trump and will vote for Biden.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Person1#: I'd like to upgrade my computer, but I'm not sure what exactly I would need. #Person2#: I'd like to have a more powerful processor, more memory, and a faster modem. #Person2#: I'd like to have a faster hard disk, to begin with. #Person1#: I'd like to have a more powerful hard disc, to begin with. #Person2#: I'd like to have a more powerful modem, to begin with. #Person1#: I'd like to have a more powerful hard disc, more memory, and a faster modem. #Person2#: I'd like to have a more powerful hard disc, to begin with. #Person2#: I'd like to have a more powerful hard disc, to begin with. #P
---------------------------------

### 3.4 Avaliação do modelo quantitativamente com a métrica ROUGE

In [61]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(tqdm(dialogues)):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to("cuda")

    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df

  0%|          | 0/10 [00:00<?, ?it/s]

,human_baseline_summaries,original_model_summaries,instruct_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,The following are the new policies:,#Person1#: I need to take a dictation for you.,#Person1#: I need to take a dictation for you.
1,In order to prevent employees from wasting tim...,#Person1: This memo is for intra-office commun...,#Person1#: I need to take a dictation for you.,#Person1#: I need to take a dictation for you.
2,Ms. Dawson takes a dictation for #Person1# abo...,The memo will be distributed to all employees ...,#Person1#: I need to take a dictation for you.,#Person1#: I need to take a dictation for you.
3,#Person2# arrives late because of traffic jam....,Taking public transport to work is a good way ...,The traffic jam at the Carrefour intersection ...,The traffic jam at the Carrefour intersection ...
4,#Person2# decides to follow #Person1#'s sugges...,#Person1: I got stuck in traffic. #Person1: It...,The traffic jam at the Carrefour intersection ...,The traffic jam at the Carrefour intersection ...
5,#Person2# complains to #Person1# about the tra...,People are choosing to take public transport t...,The traffic jam at the Carrefour intersection ...,The traffic jam at the Carrefour intersection ...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
7,#Person1# tells Kate that Masha and Hero are g...,"#Person1: It's a divorce. #Person2: Well, I ca...",Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
9,#Person1# and Brian are at the birthday party ...,"Brian, thank you for coming to the party.","#Person1#: Happy birthday, Brian. #Person2#: T...","#Person1#: Happy birthday, Brian. #Person2#: I..."


In [62]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2318199341357236, 'rouge2': 0.0754928184194043, 'rougeL': 0.20200070463228353, 'rougeLsum': 0.20477121108700053}
INSTRUCT MODEL:
{'rouge1': 0.2510566239316239, 'rouge2': 0.11535720375106562, 'rougeL': 0.229375, 'rougeLsum': 0.23316773504273502}
PEFT MODEL:
{'rouge1': 0.24089921652421653, 'rouge2': 0.11769053708439897, 'rougeL': 0.22001958689458687, 'rougeLsum': 0.22134175465057818}


In [63]:
human_baseline_summaries = df['human_baseline_summaries'].values
original_model_summaries = df['original_model_summaries'].values
instruct_model_summaries = df['instruct_model_summaries'].values
peft_model_summaries     = df['peft_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2318199341357236, 'rouge2': 0.0754928184194043, 'rougeL': 0.20200070463228353, 'rougeLsum': 0.20477121108700053}
INSTRUCT MODEL:
{'rouge1': 0.2510566239316239, 'rouge2': 0.11535720375106562, 'rougeL': 0.229375, 'rougeLsum': 0.23316773504273502}
PEFT MODEL:
{'rouge1': 0.24089921652421653, 'rouge2': 0.11769053708439897, 'rougeL': 0.22001958689458687, 'rougeLsum': 0.22134175465057818}


In [66]:
print("Porcentagem absoluta de melhora entre o PEFT MODEL em comparação a HUMAN BASELINE")
print()
print('Instruct Model')
# Instruct model
improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value, inst, ori in zip(instruct_model_results.keys(), improvement, instruct_model_results.values(),original_model_results.values() ):
    print(f'{key}: {value*100:.2f}% original = {ori} instruct = {inst}' )
print()
print('PEFT Model')
#peft model
improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value, inst, ori in zip(peft_model_results.keys(), improvement, peft_model_results.values(),original_model_results.values() ):
    print(f'{key}: {value*100:.2f}% original = {ori} instruct = {inst}' )
print()
print('Peft Model e Instruct Model')
# Peft Model over Instruct Modxel
improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value, inst, ori in zip(peft_model_results.keys(), improvement, peft_model_results.values(),instruct_model_results.values() ):
    print(f'{key}: {value*100:.2f}% original = {ori} instruct = {inst}' )

Porcentagem absoluta de melhora entre o PEFT MODEL em comparação a HUMAN BASELINE

Instruct Model
rouge1: 1.92% original = 0.2318199341357236 instruct = 0.2510566239316239
rouge2: 3.99% original = 0.0754928184194043 instruct = 0.11535720375106562
rougeL: 2.74% original = 0.20200070463228353 instruct = 0.229375
rougeLsum: 2.84% original = 0.20477121108700053 instruct = 0.23316773504273502

PEFT Model
rouge1: 0.91% original = 0.2318199341357236 instruct = 0.24089921652421653
rouge2: 4.22% original = 0.0754928184194043 instruct = 0.11769053708439897
rougeL: 1.80% original = 0.20200070463228353 instruct = 0.22001958689458687
rougeLsum: 1.66% original = 0.20477121108700053 instruct = 0.22134175465057818

Peft Model e Instruct Model
rouge1: -1.02% original = 0.2510566239316239 instruct = 0.24089921652421653
rouge2: 0.23% original = 0.11535720375106562 instruct = 0.11769053708439897
rougeL: -0.94% original = 0.229375 instruct = 0.22001958689458687
rougeLsum: -1.18% original = 0.23316773504273

In [67]:
print("Porcentagem absoluta de melhora entre o PEFT MODEL em comparação ao INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Porcentagem absoluta de melhora entre o PEFT MODEL em comparação ao INSTRUCT MODEL
rouge1: -1.02%
rouge2: 0.23%
rougeL: -0.94%
rougeLsum: -1.18%
